## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,87,90,17.27,light intensity shower rain
1,1,Comodoro Rivadavia,AR,-45.8667,-67.5000,51.66,62,0,8.05,clear sky
2,2,Bandarbeyla,SO,9.4942,50.8122,79.29,75,85,21.52,overcast clouds
3,3,Ayorou,NE,14.7308,0.9174,93.07,35,17,6.91,few clouds
4,4,Mataura,NZ,-46.1927,168.8643,51.48,72,58,4.72,light rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bandarbeyla,SO,9.4942,50.8122,79.29,75,85,21.52,overcast clouds
6,6,Guerrero Negro,MX,27.9769,-114.0611,75.49,61,1,18.32,clear sky
8,8,Half Moon Bay,US,37.4636,-122.4286,81.70,64,20,1.01,few clouds
11,11,Coruripe,BR,-10.1256,-36.1756,73.98,81,98,9.66,overcast clouds
14,14,Malakal,SS,9.5334,31.6605,73.17,87,88,1.99,overcast clouds
16,16,Sao Joao Da Barra,BR,-21.6403,-41.0511,74.03,87,10,10.11,clear sky
25,25,Sabzevar,IR,36.2126,57.6819,71.10,35,0,8.70,clear sky
31,31,Antalaha,MG,-14.9003,50.2788,70.20,84,100,10.27,overcast clouds
32,32,Chake Chake,TZ,-5.2459,39.7666,74.97,84,3,11.92,clear sky
37,37,Butte,US,39.6499,-121.5677,79.56,36,1,6.91,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

293


Unnamed: 0             293
City                   293
Country                293
Lat                    293
Lng                    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Current Description    293
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

293


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bandarbeyla,SO,9.4942,50.8122,79.29,75,85,21.52,overcast clouds
6,6,Guerrero Negro,MX,27.9769,-114.0611,75.49,61,1,18.32,clear sky
8,8,Half Moon Bay,US,37.4636,-122.4286,81.70,64,20,1.01,few clouds
11,11,Coruripe,BR,-10.1256,-36.1756,73.98,81,98,9.66,overcast clouds
14,14,Malakal,SS,9.5334,31.6605,73.17,87,88,1.99,overcast clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bandarbeyla,SO,79.29,overcast clouds,9.4942,50.8122,
6,Guerrero Negro,MX,75.49,clear sky,27.9769,-114.0611,
8,Half Moon Bay,US,81.70,few clouds,37.4636,-122.4286,
11,Coruripe,BR,73.98,overcast clouds,-10.1256,-36.1756,
14,Malakal,SS,73.17,overcast clouds,9.5334,31.6605,
16,Sao Joao Da Barra,BR,74.03,clear sky,-21.6403,-41.0511,
25,Sabzevar,IR,71.10,clear sky,36.2126,57.6819,
31,Antalaha,MG,70.20,overcast clouds,-14.9003,50.2788,
32,Chake Chake,TZ,74.97,clear sky,-5.2459,39.7666,
37,Butte,US,79.56,clear sky,39.6499,-121.5677,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key 
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bandarbeyla,SO,79.29,overcast clouds,9.4942,50.8122,JABIR HOTEL
6,Guerrero Negro,MX,75.49,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
8,Half Moon Bay,US,81.70,few clouds,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
11,Coruripe,BR,73.98,overcast clouds,-10.1256,-36.1756,Pousada e Hotel Litoral Sul
14,Malakal,SS,73.17,overcast clouds,9.5334,31.6605,Nile Palace Hotel
16,Sao Joao Da Barra,BR,74.03,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
25,Sabzevar,IR,71.10,clear sky,36.2126,57.6819,پزشکی قانونی سبزوار
31,Antalaha,MG,70.20,overcast clouds,-14.9003,50.2788,Hotel Vitasoa
32,Chake Chake,TZ,74.97,clear sky,-5.2459,39.7666,Hotel Archipelago
37,Butte,US,79.56,clear sky,39.6499,-121.5677,Meadowbrook Oaks Mobile Home Park


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))